In [48]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
from itertools import cycle, islice
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import review_preprocess
import summarize_review
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder

print("Setup Done..")

Setup Done..


In [29]:
train_file_name = 'drugsComTrain_raw.csv'
test_file_name = 'drugsComTest_raw.csv'
file_path = '../../data/'

df_test = pd.read_csv(file_path+test_file_name)
df_train = pd.read_csv(file_path+train_file_name)

In [30]:
df_test = review_preprocess.clean_reviews(df_test)
df_train = review_preprocess.clean_reviews(df_train)

Review cleanup Completed...aracters from review column--> 100.0 percentage complete
Removing row with nan values
Percentage of nan rows in dataset--> 0.55 %
Removed 295 rows with na values
Review cleanup Completed...aracters from review column--> 100.0 percentage complete
Removing row with nan values
Percentage of nan rows in dataset--> 0.56 %
Removed 899 rows with na values


In [31]:
rows,_ = df_train.shape
my_list=[]
summary_v=np.vectorize(summarize_review.summarize)
short_review = summary_v(df_train.review.values)

In [33]:
short_review_test = summary_v(df_test.review.values)
df_test["summarize"] = short_review_test

In [32]:
df_train["summarize"] = short_review

In [34]:
print(df_train[['review','summarize']].iloc[5]['review'])
print(df_train[['review','summarize']].iloc[5]['summarize'])


2nd day on 5mg started to work with rock hard erections however experianced headache lower bowel preassure. 3rd day erections would wake me up & hurt! Legankles aches   severe lower bowel preassure like you need to go 2 but cant! Enjoyed the initial rockhard erections but not at these side effects or $230 for months supply! Im 50 & work out 3Xs a week. Not worth side effects!
 3rd day erections would wake me up & hurt! Not worth side effects!


In [41]:
df_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,summarize
0,206461,Valsartan,Left Ventricular Dysfunction,It has no side effect I take it in combination...,9,20-May-12,27,It has no side effect I take it in combinatio...
1,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8,27-Apr-10,192,See how he did at school and with getting up ...
2,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5,14-Dec-09,17,And the period lasted for two weeks. The idea...
3,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8,3-Nov-15,10,This is my first time using any form of birth...
4,35696,Buprenorphine / naloxone,Opiate Dependence,Suboxone has completely turned my life around....,9,27-Nov-16,37,All that is history. Im actually sleeping bet...


In [15]:
# # Creates TF-IDF vectorizer and transforms the corpus
# vectorizer = TfidfVectorizer()
# X_train = vectorizer.fit_transform(df_train.review)

# # transforms test reviews to above vectorized format
# X_test = vectorizer.transform(df_test.review)

In [16]:
X_test.shape

(53766, 49899)

In [35]:
# Creates TF-IDF vectorizer and transforms the corpus
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_train.summarize)

# transforms test reviews to above vectorized format
X_test = vectorizer.transform(df_test.summarize)

In [36]:
X_test.shape
X_train.shape

(160398, 34136)

In [37]:
y_train = df_train.rating.copy(deep=True)
y_test = df_test.rating.copy(deep=True)


In [52]:
oe = OneHotEncoder()

y_tr = oe.fit_transform(np.array(y_train).reshape(-1,1))
y_ts = oe.transform(np.array(y_test).reshape(-1,1))

In [53]:
y_tr.shape

(160398, 10)

In [38]:
y_train

0          9
1          8
2          5
3          8
4          9
          ..
161292    10
161293     1
161294     2
161295    10
161296     9
Name: rating, Length: 160398, dtype: int64

In [39]:
%%time
# vectorizer.get_feature_names()
rfc_rating = RandomForestClassifier(n_estimators=50, random_state=42,max_features=442,n_jobs=-1,verbose=2 )
rfc_rating.fit(X_train, y_train)



building tree 1 of 50building tree 2 of 50
building tree 3 of 50
building tree 4 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50building tree 11 of 50

building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.4min


building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50
Wall time: 5min 30s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.5min finished


In [42]:
X_test.shape

(53471, 34136)

In [40]:
pred = rfc_rating.predict(X_test)

print("Accuracy: %s" % str(rfc_rating.score(X_test, y_test)))

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.3s


Accuracy: 0.7352396626208599


[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.6s finished


In [28]:
from sklearn.metrics import classification_report

target_names = ['class 0', 'class 1', 'class 2','class 3',\
                'class 4', 'class 5','class 6', 'class 7', 'class 8','class 9']
print(classification_report(y_test, pred, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.76      0.77      0.76      7299
    class 1       0.97      0.63      0.76      2334
    class 2       0.97      0.63      0.76      2205
    class 3       0.98      0.63      0.76      1659
    class 4       0.98      0.60      0.75      2710
    class 5       0.98      0.59      0.73      2119
    class 6       0.97      0.58      0.73      3091
    class 7       0.90      0.61      0.73      6156
    class 8       0.81      0.62      0.70      9177
    class 9       0.61      0.95      0.75     17016

avg / total       0.79      0.74      0.74     53766



In [44]:
model = Sequential()

# Dense-1
model.add(Dense(500, input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Dense-2
model.add(Dense(300))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Dense-3
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=Adam(lr=0.01))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               17068500  
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               150300    
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
activation_2 (Activation)    (None, 300)              

In [54]:
model_history = model.fit(X_train, y_tr, epochs=20, batch_size=256, validation_data=(X_test, y_ts))  

Train on 160398 samples, validate on 53471 samples
Epoch 1/20
160398/160398 [==============================] - 366s 2ms/step - loss: 1.7900 - acc: 0.3673 - val_loss: 1.6527 - val_acc: 0.4101
Epoch 2/20
160398/160398 [==============================] - 364s 2ms/step - loss: 1.5266 - acc: 0.4565 - val_loss: 1.5482 - val_acc: 0.4699
Epoch 3/20
160398/160398 [==============================] - 475s 3ms/step - loss: 1.2882 - acc: 0.5512 - val_loss: 1.4274 - val_acc: 0.5301
Epoch 4/20
160398/160398 [==============================] - 351s 2ms/step - loss: 1.0600 - acc: 0.6374 - val_loss: 1.3382 - val_acc: 0.5743
Epoch 5/20
160398/160398 [==============================] - 10812s 67ms/step - loss: 0.8967 - acc: 0.6981 - val_loss: 1.2798 - val_acc: 0.6058
Epoch 6/20
160398/160398 [==============================] - 356s 2ms/step - loss: 0.7809 - acc: 0.7380 - val_loss: 1.2464 - val_acc: 0.6314
Epoch 7/20
160398/160398 [==============================] - 357s 2ms/step - loss: 0.6881 - acc: 0.7727 - v